# Pinecone

In [92]:
pip install -q -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [12]:
# authenticating to Pinecone. 
# the API KEY is in .env
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
pip install --upgrade -q pinecone-client

In [ ]:
pip show pinecone-client

In [18]:
from pinecone import Pinecone,x ServerlessSpec

# Initilizing and authenticating the pinecone client
pc = Pinecone()
# pc = Pinecone(api_key='YOUR_API_KEY')

# checking authentication and read indexes in pinecone
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-jl6xhcm.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## Working with Pinecone Indexes

In [19]:
pc.list_indexes().names()

['langchain']

In [44]:
#creating pinecone indexes with serveeless
from pinecone import ServerlessSpec
index_name = 'langchain'
if index_name not in pc.list_indexes().names():
    print(f"Creating index {index_name}")
    pc.create_index(
        name=index_name, 
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
            )
        )
    print('Index created! :D')
else:
    print(f"Index {index_name} already exists")

Creating index langchain
Index created! :D


In [43]:
#deleting pinecone indexes
index_name = 'langchain'
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name}...')
    pc.delete_index(index_name)
    print('Done')
else: 
    print(f'Index {index_name} does not exist!')

Deleting index langchain...
Done


In [20]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Working with Vectors

In [34]:
#insering vectors
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_name = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [24]:
#update vectors
index.upsert(vectors=[('c', [0.5] * 1536)])

{'upserted_count': 1}

In [27]:
# fetch vectors
# index = pc.Index(index_name)
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [29]:
# delete vectors
index.delete(ids=['b', 'c'])

{}

In [35]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [31]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [39]:
# query 
query_vector = [random.random() for _ in range(1536)]
# print(query_vector)

In [37]:
# This retrieves the query_vectors of the most similar records in your index, along with their similarity scores.
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'e', 'score': 0.755557597, 'values': []},
             {'id': 'c', 'score': 0.747839808, 'values': []},
             {'id': 'b', 'score': 0.745738685, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Namespaces

In [45]:
# index.describe_index_stats()
index = pc.Index('langchain')

import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [46]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [47]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('aq')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [49]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}

In [54]:
# get specific namespace
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace', 'usage': {'read_units': 1}, 'vectors': {}}

In [53]:
# delete specific id in namespace
index.delete(ids=['x'], namespace='first-namespace')

{}

In [57]:
# delete namespace
index.delete(delete_all=True, namespace='first-namespace')

{}

## Splitting and Embedding Text Using LangChain

In [148]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [149]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
with open('files/churchill_speech.txt') as f:
    churchill_speech = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [150]:
chunks = text_splitter.create_documents([churchill_speech])
# print(chunks[0])
# print(chunks[10].page_content)
print(f'Now you have {len(chunks)}')

Now you have 300


### Embeding Cost

In [151]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00255:.6f}')

print_embedding_cost(chunks)

Total Tokens: 4820
Embedding Cost in USD: 0.012291


### Creating embeddings

In [152]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

In [153]:
vector = embeddings.embed_query(chunks[0].page_content)
print(vector)

[0.009329279355832581, 0.012539629113606017, -0.01041437719584306, 0.013753142359537666, 0.007525061761921201, -0.06292286903593268, 0.010716150838620875, 0.03695755614939609, 0.007184764894350816, 0.02986910211099426, 0.010138287565571998, 0.08475325521190123, -0.005743317446122601, 0.026581703109668123, 0.01117202017453653, -0.0012753117339436002, -0.038806719368210514, 0.015576621565845331, -0.022305515261635447, -0.03729143154817852, -0.0029888363257729285, 0.027660381343535694, -0.03667504505033708, -0.007749786626807575, -0.0165782505561602, 0.01793943988908678, -0.0027625067921814577, 0.020006905107015843, 0.028122671216916778, 0.01610311960784826, 0.017040540429541284, 0.0051108782076338035, -0.0015393630892976131, -0.026684433571760015, 0.02406478909328883, -0.018003647126386108, 0.011602207360590474, 0.04027063916322898, -0.01582060850878925, 0.03639253395127807, 0.03233464996500508, -0.03893513384280916, -0.016436995006630695, -0.014844661668336093, 0.03143575236810463, 0.02

## Inserting the Embeddings into a Pinecone Index

In [154]:
# I'm importing the necessary libraries and initializing the Pinecone client
import os
import pinecone

from langchain_community.vectorstores import Pinecone

pc = pinecone.Pinecone()

In [155]:
# deleting all indexes
indexes = pc.list_indexes().names()
for i in indexes:
    print('Deleting all indexes ... ', end='')
    pc.delete_index(i)
    print('Done')

Deleting all indexes ... Done


In [156]:
# creating an index
from pinecone import ServerlessSpec
index_name = 'churchill-speech'
if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print('Index created! 😊')
else:
    print(f'Index {index_name} already exists!')

Creating index churchill-speech
Index created! 😊


In [161]:
# processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
# inserting the embeddings into the index and returning a new Pinecone vector store object. 
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 18 May 2024 09:37:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '104', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '10304', 'x-pinecone-request-id': '225326501366153245', 'x-envoy-upstream-service-time': '43', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 3072 does not match the dimension of the index 1536","details":[]}


In [160]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Asking Questions (Similarity Search)

In [162]:
query = 'Where should we fight?'
result = vector_store.similarity_search(query)
print(result)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 18 May 2024 09:43:14 GMT', 'Content-Type': 'application/json', 'Content-Length': '104', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1170', 'x-pinecone-request-id': '717064690531565699', 'x-envoy-upstream-service-time': '39', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 3072 does not match the dimension of the index 1536","details":[]}


In [163]:
for r in result:
    print(r.page_content)
    print('-' * 50)

In [165]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Initialize the LLM with the specified model and temperature
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)

# Use the provided vector store with similarity search and retrieve top 3 results
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

# Create a RetrievalQA chain using the defined LLM, chain type 'stuff', and retriever
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)


/home/rasyidannas/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
query = 'Answer only from the provided input. Where should we fight?'
answer = chain.invoke(query)
print(answer)